In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

random.seed(777)
torch.manual_seed(777)

In [2]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [3]:
# MNIST dataset
mnist_train = dsets.MNIST(root = 'data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download = True)

mnist_test = dsets.MNIST(root = 'data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

In [4]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

In [10]:
# nn layers
linear1 = torch.nn.Linear(784, 256, bias = True)
linear2 = torch.nn.Linear(256, 256, bias = True)
linear3 = torch.nn.Linear(256, 10, bias = True)
relu = torch.nn.ReLU()

In [12]:
# initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

Parameter containing:
tensor([[-0.0039, -0.0166, -0.0415,  ...,  0.1375, -0.0556, -0.0868],
        [-0.0806,  0.0315, -0.1408,  ..., -0.1330, -0.0807,  0.0600],
        [ 0.1287,  0.0014, -0.0580,  ...,  0.0508, -0.0500, -0.1308],
        ...,
        [-0.1004, -0.0187, -0.0075,  ..., -0.0087, -0.0063,  0.0588],
        [ 0.1414,  0.1065,  0.1057,  ...,  0.1154, -0.0272, -0.0432],
        [-0.1199, -0.0441, -0.1109,  ..., -0.0223,  0.0153,  0.0347]],
       requires_grad=True)

In [13]:
# model
model = torch.nn.Sequential(linear1, relu,
                            linear2, relu,
                            linear3)

In [14]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
# training
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        # label is not one-hot encoded
        X = X.view(-1, 28*28)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

Epoch: 0001 cost = 0.243157730
Epoch: 0002 cost = 0.092084564
Epoch: 0003 cost = 0.059476588
Epoch: 0004 cost = 0.044391386
Epoch: 0005 cost = 0.032458682
Epoch: 0006 cost = 0.026315931
Epoch: 0007 cost = 0.020729860
Epoch: 0008 cost = 0.016671387
Epoch: 0009 cost = 0.018069787
Epoch: 0010 cost = 0.015915919
Epoch: 0011 cost = 0.010602980
Epoch: 0012 cost = 0.012422577
Epoch: 0013 cost = 0.009866371
Epoch: 0014 cost = 0.010768750
Epoch: 0015 cost = 0.009130007


In [16]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28*28).float()
    Y_test = mnist_test.test_labels
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy : ', accuracy.item())

Accuracy :  0.9793000221252441


C:\Users\user\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\user\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
